# Predicting Diabetes

`Diabetes.csv` extraido de [Kaggle](https://www.kaggle.com/uciml/pima-indians-diabetes-database). Tenemos varias preguntas: 

* qué información está más correlacionada con un diagnóstico positivo, y si solo podemos hacerle dos preguntas a un paciente, 
    + qué deberíamos preguntar 
    + cómo le daríamos un riesgo de ser diagnosticado.

Esta es una base de datos de Machine Learning, y normalmente solo extraeríamos características, alimentaríamos a un algoritmo ML y nos sentaríamos y relajaríamos. Pero nos *ensuciamos* las manos para que puedas aprender más.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
df_original = pd.read_csv("Diabetes.csv")
df_original.head()

In [ ]:
df = df_original.drop(["Pregnancies", "DiabetesPedigreeFunction"], axis=1)
pd.plotting.scatter_matrix(df, figsize=(6,6), s=2);

Primero respondamos la pregunta: *¿Qué información está más correlacionada con la diabetes?*

In [ ]:
df.corr()

In [ ]:
import seaborn as sb
sb.heatmap(df.corr(), annot=True, cmap="viridis");

De las correlaciones anteriores, un enfoque ingenuo podría ser tomar los principales observables correlacionados y profundizar en ellos. En nuestro caso, Glucosa, IMC y Edad.

In [ ]:
df2 = df[["Glucose", "BMI", "Age", "Outcome"]]
df2.head()

In [ ]:
pd.plotting.scatter_matrix(df2, s=4);

¡Suerte que visualizamos esto! Mire los histogramas de Glucosa e IMC: los picos en cero son indicativos de que el conjunto de datos está usando el valor cero cuando no hay datos. Filtrémoslos. Y también dejemos de lado la edad para mantener esto más corto.

In [ ]:
df3 = df2.loc[(df2["Glucose"] > 1) & (df2["BMI"] > 1), ["Glucose", "BMI", "Outcome"]]
df3.head()

In [ ]:
df_y = df3.loc[df3["Outcome"] == 1, ["Glucose", "BMI"]]
df_n = df3.loc[df3["Outcome"] == 0, ["Glucose", "BMI"]]

plt.scatter(df_y["Glucose"], df_y["BMI"], c="w", s=1, label="Tiene Diabetes")
plt.scatter(df_n["Glucose"], df_n["BMI"], s=1, label="No Tiene Diabetes")
plt.legend(loc=2)
plt.xlabel("Glucose")
plt.ylabel("BMI");

In [ ]:
pd.plotting.scatter_matrix(df_y)
pd.plotting.scatter_matrix(df_n);

Así que no es perfecto, pero probablemente podamos hacer un buen trabajo aproximando ambas distribuciones como gaussianas.

Agreguemos también a la mezcla un paciente de prueba que ingresa, con Glucosa de 140 e IMC de 35. ¿Cuál es su probabilidad de ser diagnosticado?

In [ ]:
from scipy.stats import multivariate_normal as mn
prob_test = []
test_point = [140, 35]
x, y = np.meshgrid(np.linspace(50, 200, 20), np.linspace(10, 50, 20), indexing='ij')
points = np.dstack((x.flatten(), y.flatten()))

In [ ]:
for d, l in zip([df_y, df_n], ["Sí", "No"]):
    is_yes = l == "Sí"
    mean = np.mean(d)
    cov = np.cov(d, rowvar=0)
    probs = mn.pdf(points, mean, cov).reshape(x.shape)
    prob_test.append(mn.pdf(test_point, mean, cov))
    plt.contour(x, y, probs, 
                cmap="viridis" if is_yes else "magma", 
                linestyles="-" if is_yes else "--")
    marker = "." if is_yes else "+"
    color = "g" if is_yes else "y"
    plt.scatter(d["Glucose"], d["BMI"], c=color, marker=marker, s=20, alpha=0.3, label=l)
plt.axvline(test_point[0], ls=":", lw=1)
plt.axhline(test_point[1], ls=":", lw=1)
plt.legend(loc=2)
plt.xlabel("Glucose")
plt.ylabel("BMI");

In [ ]:
num_y = df_y.shape[0]
num_n = df_n.shape[0]

prob_diagnosis = num_y * prob_test[0] / (num_y * prob_test[0] + num_n * prob_test[1])
print(f"La probabilidad de diagnóstico positivo es {100 * prob_diagnosis:.2f}%")

Esto puede parecer extraño. Dado que el paciente de prueba está justo en el máximo de nuestro modelo para los pacientes con diabetes, seguramente debería haber una mayor posibilidad, ¿verdad? ¡No!

La razón es que, aunque la probabilidad de distribución es mayor, hay muchos más pacientes sin diabetes que con diabetes. Solo podemos comparar directamente las dos distribuciones si tienen la misma probabilidad (el mismo número de personas con y sin). Este rara vez es el caso, por lo que tenemos que ponderarlos. En una formalización bayesiana, estamos modificando nuestro modelo a priori.